# Byte-Pair Encoding

## About

Byte-Pair Encoding (BPE) was initially developed as an algorithm to compress texts, and then used by OpenAI for tokenization when pretraining the GPT model. It’s used by a lot of Transformer models, including GPT, GPT-2, RoBERTa, BART, and DeBERTa.

## Training

BPE training starts by computing the unique set of words used in the corpus (after the normalization and pre-tokenization steps are completed), then building the vocabulary by taking all the symbols used to write those words.

For real-world cases, base vocabulary will contain all the ASCII characters, at the very least, and probably some Unicode characters as well. If an example you are tokenizing uses a character that is not in the training corpus, that character will be converted to the unknown token.

Now we add new tokens until the desired vocabulary size is reached by learning merges, which are rules to merge two elements of the existing vocabulary together into a new one.

At any step during the tokenizer training, the BPE algorithm will search for the most frequent pair of existing tokens.

### Example

As a very simple example, let’s say our corpus uses these five words:

`"hug", "pug", "pun", "bun", "hugs"`

The base vocabulary will then be ["b", "g", "h", "n", "p", "s", "u"].

let’s assume the words had the following frequencies:

`("hug", 10), ("pug", 5), ("pun", 12), ("bun", 4), ("hugs", 5)`

meaning "hug" was present 10 times in the corpus, "pug" 5 times, "pun" 12 times, "bun" 4 times, and "hugs" 5 times. We start the training by splitting each word into characters (the ones that form our initial vocabulary) so we can see each word as a list of tokens:

`("h" "u" "g", 10), ("p" "u" "g", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "u" "g" "s", 5)`

Then we look at pairs. The pair ("h", "u") is present in the words "hug" and "hugs", so 15 times total in the corpus. It’s not the most frequent pair, though: that honor belongs to ("u", "g"), which is present in "hug", "pug", and "hugs", for a grand total of 20 times in the vocabulary.

Thus, the first merge rule learned by the tokenizer is ("u", "g") -> "ug", which means that "ug" will be added to the vocabulary, and the pair should be merged in all the words of the corpus. At the end of this stage, the vocabulary and corpus look like this:

`Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug"]`

`Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "u" "n", 12), ("b" "u" "n", 4), ("h" "ug" "s", 5)`

Now we have some pairs that result in a token longer than two characters: the pair ("h", "ug"), for instance (present 15 times in the corpus). The most frequent pair at this stage is ("u", "n"), however, present 16 times in the corpus, so the second merge rule learned is ("u", "n") -> "un". Adding that to the vocabulary and merging all existing occurrences leads us to:

`Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un"]`

`Corpus: ("h" "ug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("h" "ug" "s", 5)`

Now the most frequent pair is ("h", "ug"), so we learn the merge rule ("h", "ug") -> "hug", which gives us our first three-letter token. After the merge, the corpus looks like this:

`Vocabulary: ["b", "g", "h", "n", "p", "s", "u", "ug", "un", "hug"]`

`Corpus: ("hug", 10), ("p" "ug", 5), ("p" "un", 12), ("b" "un", 4), ("hug" "s", 5)`

And we continue like this until we reach the desired vocabulary size.

## Tokenization Algorithm

Tokenization follows the training process closely, in the sense that new inputs are tokenized by applying the following steps:

1.   Normalization
2.   Pre-tokenization
3.   Splitting the words into individual characters
4.   Applying the merge rules learned in order on those splits

Let’s take the example we used during training, with the three merge rules learned:

`("u", "g") -> "ug"`

`("u", "n") -> "un"`

`("h", "ug") -> "hug"`

The word "bug" will be tokenized as ["b", "ug"]. "mug", however, will be tokenized as ["[UNK]", "ug"] since the letter "m" was not in the base vocabulary. Likewise, the word "thug" will be tokenized as ["[UNK]", "hug"]: the letter "t" is not in the base vocabulary, and applying the merge rules results first in "u" and "g" being merged and then "h" and "ug" being merged.

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
sentence = "This is a sample sentence."
tokens = tokenizer(sentence)
print(tokens)

{'input_ids': [1212, 318, 257, 6291, 6827, 13], 'attention_mask': [1, 1, 1, 1, 1, 1]}


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mshojaei77/PersianBPETokenizer")
test_sentence = "سلام، چطور هستید؟ امیدوارم روز خوبی داشته باشید"
tokens = tokenizer.tokenize(test_sentence)
print("Tokens:", tokens)
encoded = tokenizer(test_sentence)
print("Input IDs:", encoded["input_ids"])
print("Decoded:", tokenizer.decode(encoded["input_ids"]))


Tokens: ['سلام', '،', 'چطور', 'هستید', '؟', 'امیدوارم', 'روز', 'خوبی', 'داشته', 'باشید']
Input IDs: [1, 3007, 241, 3167, 3883, 243, 4526, 2063, 2750, 2297, 2863, 2]
Decoded: " سلام ، چطور هستید ؟ امیدوارم روز خوبی داشته باشید #


In [ ]:
from collections import defaultdict

# corpus = [
#     "This is the Hugging Face Course.",
#     "This chapter is about tokenization.",
#     "This section shows several tokenizer algorithms.",
#     "Hopefully, you will be able to understand how they are trained and generate tokens.",
# ]
corpus = ["این یک عبارت برای تست روش مورد بررسی است.",
          "در این مدل سعی در توکنایز کردن این عبارت داریم."
]

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'این': 3, 'یک': 1, 'عبارت': 2, 'برای': 1, 'تست': 1, 'روش': 1, 'مورد': 1, 'بررسی': 1, 'است': 1, '.': 2, 'در': 2, 'مدل': 1, 'سعی': 1, 'توکنایز': 1, 'کردن': 1, 'داریم': 1})


In [ ]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

vocab = ["<|endoftext|>"] + alphabet.copy()
splits = {word: [c for c in word] for word in word_freqs.keys()}


print(alphabet)
print(splits)

['.', 'ا', 'ب', 'ت', 'د', 'ر', 'ز', 'س', 'ش', 'ع', 'ل', 'م', 'ن', 'و', 'ک', 'ی']
{'این': ['ا', 'ی', 'ن'], 'یک': ['ی', 'ک'], 'عبارت': ['ع', 'ب', 'ا', 'ر', 'ت'], 'برای': ['ب', 'ر', 'ا', 'ی'], 'تست': ['ت', 'س', 'ت'], 'روش': ['ر', 'و', 'ش'], 'مورد': ['م', 'و', 'ر', 'د'], 'بررسی': ['ب', 'ر', 'ر', 'س', 'ی'], 'است': ['ا', 'س', 'ت'], '.': ['.'], 'در': ['د', 'ر'], 'مدل': ['م', 'د', 'ل'], 'سعی': ['س', 'ع', 'ی'], 'توکنایز': ['ت', 'و', 'ک', 'ن', 'ا', 'ی', 'ز'], 'کردن': ['ک', 'ر', 'د', 'ن'], 'داریم': ['د', 'ا', 'ر', 'ی', 'م']}


In [ ]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

pair_freqs = compute_pair_freqs(splits)

for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

merges = {("ی", "ا"): "ای"}
vocab.append("ای")

def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

splits = merge_pair("ی", "ا", splits)
print(splits["این"])

('ا', 'ی'): 5
('ی', 'ن'): 3
('ی', 'ک'): 1
('ع', 'ب'): 2
('ب', 'ا'): 2
('ا', 'ر'): 3
('ا', 'ی') 5
['ا', 'ی', 'ن']


In [ ]:
vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

print(merges)
print(vocab)

{('ی', 'ا'): 'ای', ('ا', 'ی'): 'ای', ('ای', 'ن'): 'این', ('ا', 'ر'): 'ار', ('ع', 'ب'): 'عب', ('عب', 'ار'): 'عبار', ('عبار', 'ت'): 'عبارت', ('ب', 'ر'): 'بر', ('س', 'ت'): 'ست', ('ر', 'د'): 'رد', ('د', 'ر'): 'در', ('ی', 'ک'): 'یک', ('بر', 'ای'): 'برای', ('ت', 'ست'): 'تست', ('ر', 'و'): 'رو', ('رو', 'ش'): 'روش', ('م', 'و'): 'مو', ('مو', 'رد'): 'مورد', ('بر', 'ر'): 'برر', ('برر', 'س'): 'بررس', ('بررس', 'ی'): 'بررسی', ('ا', 'ست'): 'است', ('م', 'د'): 'مد', ('مد', 'ل'): 'مدل', ('س', 'ع'): 'سع', ('سع', 'ی'): 'سعی', ('ت', 'و'): 'تو', ('تو', 'ک'): 'توک', ('توک', 'ن'): 'توکن', ('توکن', 'ای'): 'توکنای', ('توکنای', 'ز'): 'توکنایز', ('ک', 'رد'): 'کرد'}
['<|endoftext|>', '.', 'ا', 'ب', 'ت', 'د', 'ر', 'ز', 'س', 'ش', 'ع', 'ل', 'م', 'ن', 'و', 'ک', 'ی', 'ای', 'ای', 'ای', 'این', 'ار', 'عب', 'عبار', 'عبارت', 'بر', 'ست', 'رد', 'در', 'یک', 'برای', 'تست', 'رو', 'روش', 'مو', 'مورد', 'برر', 'بررس', 'بررسی', 'است', 'مد', 'مدل', 'سع', 'سعی', 'تو', 'توک', 'توکن', 'توکنای', 'توکنایز', 'کرد']


In [ ]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

tokenize("این یک عبارت است.")

['این', 'یک', 'عبارت', 'است', '.']